In [1]:
import pandas as pd
import numpy as np
from sklearn import feature_extraction
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem.snowball import SnowballStemmer
from nltk import collocations
import re, string
import matplotlib
import matplotlib.pyplot as plt
from sklearn import preprocessing
from scipy import sparse
from sklearn import linear_model
plt.rcParams['figure.figsize'] = [16, 6]

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mdleiton/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/mdleiton/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [190]:
def getxy(row_s, row_f, feature_cols=['content',"sin_stopwords", 'followers', 'following', 'retweet',"n_mentioned","n_hashtags"], label_col=['troll']):
    return df[feature_cols][row_s:row_f], df[label_col][row_s:row_f]

In [202]:
df = pd.read_csv('alldataset_sentimental.csv', sep=",", encoding='utf-8')
users_copy = df[df["troll"] == False].copy()
trolls_copy =  df[df["troll"] == True].copy()

In [203]:
df.sample(5).head()

author                                            content  \
25386         el_comodo  RT @MashiRafael: Esto habla muy bien de Tuárez...   
238852       Ecuagallos  Mandar "buenas personas" al infierno (Cpccs) p...   
335910       MiguelEras         RT @Thrones_Memes: https://t.co/FoEke3RHON   
91664   IsabelC19062013  RT @JoanibManzano: ¡Qué VERGONZOSO e INDIGNANT...   
92434   IsabelC19062013  RT @tahigue29: Lenin moreno visita Machala mañ...   

       followers following retweet           created_at reply_to  n_mentioned  \
25386         46       281    True  2019-07-13 12:02:30      NaN          1.0   
238852       949      1280   False  2019-03-04 20:36:39      NaN          0.0   
335910       708       450    True  2018-09-11 16:59:46      NaN          1.0   
91664        427       508    True  2019-07-09 14:54:31      NaN          2.0   
92434        427       508    True  2019-06-20 22:39:18      NaN          1.0   

       n_hashtags       user_mentioned hashtags  troll emoticones  \
25386           0          MashiRafael      NaN   True        NaN   
238852          0                  NaN      NaN  False        NaN   
335910          0        Thrones_Memes      NaN  False        NaN   
91664           0  JoanibManzano;Lenin      NaN   True          ¡   
92434           0            tahigue29      NaN   True          ¡   

                                            sin_stopwords  polarity  
25386       habla bien tuárez significa haciendo correcto       0.0  
238852  mandar buenas personas infierno cpccs enfriarl...       0.0  
335910                                                NaN       0.0  
91664   vergonzoso indignante ver maldito impostor sec...       0.0  
92434   lenin moreno visita machala mañana viernes ven...       0.0

In [204]:
df.shape

(395149, 15)

In [205]:
df = df[['content', 'followers', 'following', 'retweet',"n_mentioned","troll",'sin_stopwords',"n_hashtags"]].dropna()

In [206]:
df.isnull().values.any()

False

In [207]:
df.shape

(374753, 8)

In [208]:
df['retweet'].value_counts()

False    183555
True     127114
True      38653
False     25431
Name: retweet, dtype: int64

In [209]:
df['troll'].value_counts()

False    210906
True     163847
Name: troll, dtype: int64

In [210]:
df = df.sample(frac=1)

In [211]:
X_train, y_train = getxy(0,299802)

In [212]:
X_test, y_test = getxy(299803,316119)

In [213]:
X_train.shape, y_train.shape

((299802, 7), (299802, 1))

In [214]:
X_test.shape, y_test.shape

((16316, 7), (16316, 1))

### Tokenize words

In [217]:
vocab_size=5000
tokenizer=feature_extraction.text.CountVectorizer(stop_words=stopwords_set, max_features=vocab_size)
tokenizer=tokenizer.fit(df['sin_stopwords'])

In [218]:
X_train_tok=tokenizer.transform(X_train['content'])
X_test_tok=tokenizer.transform(X_test['content'])

### Standarize followers/following

In [219]:
scaler = preprocessing.StandardScaler().fit(X_train[['followers','following',"n_mentioned","n_hashtags"]])

In [220]:
'rand means and scales: {}, {}'.format(scaler.mean_, scaler.scale_)

'rand means and scales: [5.36877281e+03 9.02260782e+02 1.34134862e+00 3.59674052e-01], [3.01723317e+04 2.77200829e+03 1.73109190e+00 1.19261143e+00]'

In [221]:
col_to_std = ['followers', 'following',"n_mentioned","n_hashtags"]
X_train[col_to_std]=scaler.transform(X_train[col_to_std])
X_test[col_to_std]=scaler.transform(X_test[col_to_std])

In [222]:
X_train[col_to_std].head()

followers  following  n_mentioned  n_hashtags
208638   6.783408  10.333569    -0.197187   -0.301585
256981  -0.060777  -0.301680    -0.197187    0.536911
160521  -0.176910  -0.257669    -0.197187   -0.301585
172063  -0.146749   0.258202    -0.774857   -0.301585
167161  -0.177108  -0.283282    -0.197187   -0.301585

### Binarize outcome

In [223]:
y_train.head()

troll
208638  False
256981  False
160521   True
172063  False
167161   True

In [224]:
bool_to_bin = lambda x: 1 if x else 0
y_train['troll'] = y_train['troll'].apply(bool_to_bin)
y_test['troll'] = y_test['troll'].apply(bool_to_bin)

# binarize retweet colum
X_train['retweet'] = X_train['retweet'].apply(bool_to_bin)
X_test['retweet'] = X_test['retweet'].apply(bool_to_bin)

### Concatenate features

In [226]:
def concatenate_features(tok_matrix, data_df):
    """ concatenate the tokenized matrix (scipy.sparse) with other features """
    sparse_cols = sparse.csr_matrix(data_df[['followers', 'following', 'retweet',"n_mentioned","n_hashtags"]])
    combined = sparse.hstack([tok_matrix, sparse_cols])
    return combined

In [227]:
X_train_combined = concatenate_features(X_train_tok, X_train)
X_test_combined = concatenate_features(X_test_tok, X_test)

In [228]:
X_train_combined.shape

(299802, 5005)

## Training the model - Logistic Regresion

In [229]:
logic_model = linear_model.LogisticRegression().fit(X_train_combined, y_train['troll'])

/media/mdleiton/Respaldo/repositorios/AED/env/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [230]:
logic_model.score(X_train_combined, y_train['troll'])

0.855194428322693

In [231]:
logic_model.score(X_test_combined, y_test['troll'])

0.8494729100269673

## Creating Neural Net

In [233]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

Using TensorFlow backend.


In [234]:
X_train_combined.shape[1:]

(5005,)

In [235]:
model = Sequential()  # 5003 inputs
model.add(Dense(1024, activation='relu', input_shape=X_train_combined.shape[1:])) # first layer
model.add(Dropout(0.7))
model.add(Dense(512, activation='relu'))  # second layer 1024 inputs
model.add(Dropout(0.7))
model.add(Dense(128, activation='relu'))  # third layer  512 inputs
model.add(Dense(1, activation='sigmoid'))  # last layer





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [236]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              5126144   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               65664     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 5,716,737
Trainable params: 5,716,737
Non-trainable params: 0
____________________________________________

In [237]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [238]:
# converting test scipy.sparse matrix to numpy
X_test_matrix = X_test_combined.todense()
X_train_matrix = X_train_combined.todense()

In [239]:
score = model.evaluate(X_test_matrix, y_test['troll'], verbose=0)
accuracy = 100*score[1]

print('Precisión en el conjunto de prueba: %.4f%%' % accuracy)

Precisión en el conjunto de prueba: 43.1049%


In [242]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='trolls.model.best.hdf5', verbose=1, save_best_only=True)
hist = model.fit(X_train_matrix, y_train['troll'], batch_size=1024, epochs=30, validation_split=0.2, callbacks=[checkpointer], verbose=1, shuffle=True)

Train on 239841 samples, validate on 59961 samples
Epoch 1/30
239841/239841 [==============================] - 60s 250us/step - loss: 0.2273 - acc: 0.9088 - val_loss: 0.2964 - val_acc: 0.8801

Epoch 00001: val_loss improved from inf to 0.29638, saving model to trolls.model.best.hdf5
Epoch 2/30
239841/239841 [==============================] - 66s 277us/step - loss: 0.1943 - acc: 0.9231 - val_loss: 0.3117 - val_acc: 0.8801

Epoch 00002: val_loss did not improve from 0.29638
Epoch 3/30
239841/239841 [==============================] - 65s 273us/step - loss: 0.1701 - acc: 0.9335 - val_loss: 0.3262 - val_acc: 0.8805

Epoch 00003: val_loss did not improve from 0.29638
Epoch 4/30
239841/239841 [==============================] - 66s 276us/step - loss: 0.1514 - acc: 0.9413 - val_loss: 0.3399 - val_acc: 0.8794

Epoch 00004: val_loss did not improve from 0.29638
Epoch 5/30
239841/239841 [==============================] - 63s 264us/step - loss: 0.1393 - acc: 0.9461 - val_loss: 0.3455 - val_acc: 0.8

In [243]:
model.load_weights('trolls.model.best.hdf5')
score = model.evaluate(X_test_matrix, y_test['troll'], verbose=0)
accuracy = 100*score[1]

# mostrar la precisión en prubea
print('Precisión durante la prueba: %.4f%%' % accuracy)

Precisión durante la prueba: 87.4969%
